In [1]:
# Configuración inicial
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Verificar si la clave API de Groq está presente
if 'GROQ_API_KEY' not in os.environ:
    raise ValueError("La clave API de Groq no está configurada en el archivo .env")


In [2]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage, FunctionMessage

ModuleNotFoundError: No module named 'langchain_groq'

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
system_prompt_template = """Eres un asistente útil y servicial. 
    Tu objetivo es proporcionar información precisa y ayudar en lo que sea necesario. 
    Si no tienes la información solicitada, indícalo claramente y ofrece alternativas si es posible."""

In [ ]:
def chatbot(state: State):
    system_prompt = system_prompt_template
    
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [response]}

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")

graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Correr el chatbot

In [ ]:
def add_message(message_content: str, is_human: bool, previous_state: State = None) -> State:
    if previous_state is None:
        previous_state = {"messages": []}
    new_message = HumanMessage(content=message_content) if is_human else AIMessage(content=message_content)
    return {
        "messages": previous_state["messages"] + [new_message]
    }

In [ ]:
result = {'messages': []}

In [ ]:
state = add_message("Y que podes hacer?", is_human=True, previous_state=result)

In [ ]:
state

In [ ]:
result = graph.invoke(state)
result